# Load packages

In [ ]:
import ThermoPhysicalModeling
import Downloads
using StaticArrays
using LinearAlgebra

# Download files

To compare with Kanamaru et al. (2021), plase use the public shape models `SHAPE_SFM_49k_v20180804.obj`.

In [ ]:
path_obj = "SHAPE_SFM_49k_v20180804.obj"
if !isfile(path_obj)
    url_obj = "https://data.darts.isas.jaxa.jp/pub/hayabusa2/paper/Watanabe_2019/SHAPE_SFM_49k_v20180804.obj"
    Downloads.download(url_obj, path_obj)
end

# Load shape model

In [ ]:
path_jld = splitext(path_obj)[1]*".jld2"
if isfile(path_jld)
    shape = ThermoPhysicalModeling.ShapeModel(path_jld)
    println("A preprocessed shape file was loaded ($path_jld).")
else
    shape = ThermoPhysicalModeling.ShapeModel(path_obj; scale=1000, find_visible_facets=true, save_shape=true)
    println("An OBJ shape file was loaded ($path_obj).")
end

# Replicate the result of Kanamaru et al. (2021)
Thermophysical model in Kanamaru et al. (2021):
- 3D shape of asteroid Ryugu
- Self-shadowing
- Zero-conductivity
- Ignore reabsorption of scttering and radiation

In [ ]:
"""
    solar_irradiation(distance) -> solar_irrad

Calculate the solar irradiation on the body

# Parameter
- `rₕ` : heliocentric distance of the body [m]

# Return
- `F☉` : solar irradiation [W/m^2]
"""
solar_irradiation(rₕ) = ThermoPhysicalModeling.SOLAR_CONST / (rₕ / ThermoPhysicalModeling.AU)^2


"""
    solar_condition(orbit, spin, time) -> Φ, r_sun

Get the solar irradition and the direction of the Sun

# Parameters
- `orbit` : Orbital elements
- `spin`  : Spin parameters
- `time`  : Epoch in seconds

# Returns
- `F☉` : solar irradiation [W/m^2]
- `r̂☉` : solar direction in the body-fixed frame
"""
function solar_condition(orbit, spin, time)
    u = ThermoPhysicalModeling.solveKeplerEquation2(orbit, time)
    r = ThermoPhysicalModeling.get_r(orbit, u)
    F☉ = solar_irradiation(norm(r))

    r̂☉ = normalize(r) * -1  # Shift the origin from the sun to the body

    spin_phase = spin.ω * time
    r̂☉ = ThermoPhysicalModeling.orbit_to_body(r̂☉, spin.γ, spin.ε, spin_phase)

    F☉, r̂☉
end


"""
    sum_torque_over_surface(shape, F☉, r̂☉) -> τ

# Parameters
- `shape` : Shape model
- `F☉`    : Solar irradiation [W/m^2]
- `r̂☉`    : Direction of the sun in the body-fixed frame (normalized)

# Return
- `τ`
"""
function sum_torque_over_surface(shape, F☉, r̂☉)
    τ = MVector(0., 0., 0.)  # YORP torque

    for facet in shape.facets
        Ψ = facet.normal ⋅ r̂☉  # cosine of the Sun illumination angle
        if Ψ > 0  # daytime hemisphere of the body
            if ThermoPhysicalModeling.isIlluminated(facet, r̂☉, shape.facets)
                df = Ψ * facet.area * facet.normal  # force on each facet
                dτ = facet.center × df              # torque on each facet
                τ .+= dτ
            end
        end
    end
    τ *= - 2/3 * F☉ / ThermoPhysicalModeling.c₀
    return SVector(τ)
end


"""
    net_torque(shape, orbit, spin, times) -> τ̄

Average YORP torque over given time steps
"""
function net_torque(shape, orbit, spin, times)
    τ̄ = MVector(0., 0., 0.)  # net YORP torque

    for time in times
        spin_phase = spin.ω * time
        F☉, r̂☉ = solar_condition(orbit, spin, time)
        τ = sum_torque_over_surface(shape, F☉, r̂☉)
        τ = ThermoPhysicalModeling.body_to_orbit(τ, spin.γ, spin.ε, spin_phase)

        τ̄ .+= τ
    end
    τ̄ /= length(times)
end

In [ ]:
orbit = ThermoPhysicalModeling.OrbitalElements(ThermoPhysicalModeling.RYUGU)

In [ ]:
spin = ThermoPhysicalModeling.SpinParams(ThermoPhysicalModeling.RYUGU, orbit)

In [ ]:
Δt = spin.P / 72
times = collect(0:Δt:orbit.P);

YORP torque averaged over an orbit cycle. In this setting, a laptop computer takes about an hour to calculate.

In [ ]:
τ̄ = net_torque(shape, orbit, spin, times)

In [ ]:
MOI = 4.039541372643629e16  # Moment of inertia of Ryugu

## Orbitally averaged torque [N ⋅ m]
τ̄_ω = ThermoPhysicalModeling.getτω(τ̄, spin)
τ̄_ε = ThermoPhysicalModeling.getτε(τ̄, spin)
τ̄_ψ = ThermoPhysicalModeling.getτψ(τ̄, spin)

## Acceleration rate [rad/sec/sec]
ω̇  = τ̄_ω / MOI
ωε̇ = τ̄_ε / MOI
ωψ̇ = τ̄_ψ / MOI

## Acceleration rate [deg/day/day]
ω̇  = rad2deg(ω̇)  * (3600*24)^2
ωε̇ = rad2deg(ωε̇) * (3600*24)^2
ωψ̇ = rad2deg(ωψ̇) * (3600*24)^2;

In [ ]:
@show ω̇
@show ωε̇
@show ωψ̇;

See the result of `SHAPE_SFM_49k_v20180804.obj` in Table 3 in Kanamaru et al. (2021). Because of a change of the libarary, it might make a difference of a few percent.

(ω̇, ωε̇, ωψ̇) = (-3.531e-6, 1.667e-6, 3.973e-5)